# Часть 4. Excel/Python на выбор

<div style="border:solid black 2px; padding: 20px">

Работа с таблицами:
* t_products
* t_whs
* t_sale

</div>

<div style="border:solid black 2px; padding: 20px">

## Загрузка данных.

</div>

In [1]:
import pandas as pd

In [2]:
file_1 = 't_sale.xlsx' 
file_2 = 't_products.xlsx'
file_3 = 't_whs.xlsx' 
sale = pd.read_excel(file_1)
products = pd.read_excel(file_2)
whs = pd.read_excel(file_3)

In [3]:
display(sale.head(2))
display(products.head(2))
display(whs.head(2))

,month,whs_id,product_id,quantity,sale,sale_cogs
0,2,448,6553,9,958.950562,728.28
1,1,54,8370,2,121.800000,92.58


,product_id,level1,level2,name
0,1,Toys for children 0 +,Interactive toys,Пирамидка
1,2,Toys for children 0 +,Interactive toys,Подушка-игрушка1


,whs_id,name,city_id,city
0,1,СПБ 1,1,Санкт-Петербург
1,2,СПБ 2,1,Санкт-Петербург


<div style="border:solid black 2px; padding: 20px">

## Объединение таблиц.

</div>

In [4]:
data = sale.merge(products, how='left', on='product_id')
data = data.merge(whs, how='left', on='whs_id')
data.head(2)

,month,whs_id,product_id,quantity,sale,sale_cogs,level1,level2,name_x,name_y,city_id,city
0,2,448,6553,9,958.950562,728.28,Crafts and Painting kits,Stickers,Наклейка11,СПБ 55,1,Санкт-Петербург
1,1,54,8370,2,121.800000,92.58,Swimwear,Swimsuit,Трикини_2,МСК 17,2,Москва


<div style="border:solid black 2px; padding: 20px">

## Какая Выручка была в Феврале в Москве?

</div>

In [5]:
data.query('city == "Москва" and month == 2')\
    .pivot_table(index=['city', 'month'], values='sale', aggfunc='sum').round()

,,sale
city,month,
Москва,2,41670934.0


<div style="border:solid green 2px; padding: 20px">

Выручка в феврале в Москве: 41.670.934 руб.

</div>

<div style="border:solid black 2px; padding: 20px">

## По какой категории были наибольшие продажи в шт в Санкт-Петербурге в Январе?

</div>

In [6]:
top_level = data.query('city == "Санкт-Петербург" and month == 1')\
           .pivot_table(index=['city', 'month', 'level1'], values=['quantity', 'sale'], aggfunc='sum')\
           .reset_index().sort_values(by='quantity', ascending=False).round()
top_level.head(1)

,city,month,level1,quantity,sale
4,Санкт-Петербург,1,Bijou,52300,3353660.0


<div style="border:solid green 2px; padding: 20px">

В январе в Санкт-Петербурге были наибольшие продажи в шт в категории Bijou (Ювелирные изделия).

</div>

<div style="border:solid black 2px; padding: 20px">

## Какая доля продаж в общем обороте Санкт-Петербурга в январе по категории из пункта 2?

</div>

In [7]:
top_level['percent_quantity'] = round((top_level['quantity']
                       /top_level['quantity'].sum()) * 100, 1)
top_level['percent_quantity'] = top_level['percent_quantity'].map('{:,.1f}%'.format)

top_level['percent_sale'] = round((top_level['sale']
                       /top_level['sale'].sum()) * 100, 1)
top_level['percent_sale'] = top_level['percent_sale'].map('{:,.1f}%'.format)

top_level.head(1)

,city,month,level1,quantity,sale,percent_quantity,percent_sale
4,Санкт-Петербург,1,Bijou,52300,3353660.0,5.3%,4.5%


<div style="border:solid green 2px; padding: 20px">

Доля продаж в общем обороте Санкт-Петербурга в январе по категории Bijou:
* в рублях 4.5\%
* в штуках 5.3\%

</div>

<div style="border:solid black 2px; padding: 20px">

## Какая позиция имеет наибольшую цену в категории из пункта 2 в январе в Санкт-Петербурге?

</div>

In [8]:
data_price = data.query('city == "Санкт-Петербург" and month == 1 and level1 == "Bijou"')
data_price = data_price.pivot_table(index = ['level1', 'level2', 'product_id', 'name_x'],
                                   values=['sale', 'quantity'], aggfunc='sum')\
                      .reset_index()
data_price['avg_price'] = round(data_price['sale'] / data_price['quantity'], 0)
data_price['sale'] = data_price['sale'].round(0)

data_price.sort_values(by = 'avg_price', ascending=False).head(1)

,level1,level2,product_id,name_x,quantity,sale,avg_price
85,Bijou,Ring,920,Кольцо_21111111111,12,14894.0,1241.0


<div style="border:solid green 2px; padding: 20px">

В Санкт-Петербурге в январе в категории Bijou позиция с наибольшей ценой:

Кольцо_21111111111: 1241 руб.

</div>

<div style="border:solid black 2px; padding: 20px">

## Какая маржа в руб и % по позиции из пункта 4 в Санкт-Петербурге в Январе ?

</div>

In [9]:
margin = data.query('city == "Санкт-Петербург" and month == 1 and name_x == "Кольцо_21111111111"')\
             .pivot_table(index=['name_x', 'product_id'], values=['sale', 'sale_cogs', 'quantity'], aggfunc='sum')\
             .round(1)
margin['margin'] = margin['sale'] - margin['sale_cogs'].round(2)
margin['margin_%'] = round(margin['margin'] / margin['sale'] * 100, 2)
margin['margin_of_one'] = round(margin['margin'] / margin['quantity'], 2)
margin

,,quantity,sale,sale_cogs,margin,margin_%,margin_of_one
name_x,product_id,,,,,,
Кольцо_21111111111,920,12,14893.9,11296.7,3597.2,24.15,299.77


<div style="border:solid green 2px; padding: 20px">

В Санкт-Петербурге в январе маржа товара id 920 Кольцо_21111111111:
* 3597 руб.
* 24.15\%
* за 1 шт 299.77 руб.

</div>

<div style="border:solid black 2px; padding: 20px">

## Как изменится цена продажи позиции из пункта 4, если увеличить маржу на 5\%?

</div>

In [10]:
# ниже расчет средней цены, по которой надо продавать товар, чтобы маржа увеличилась на 5%
growth = 0.05
margin['margin_of_one_new'] = round(margin['margin_of_one'] * (1 + growth), 2)
margin['price_new'] = round(margin['margin_of_one_new']
                            + (margin['sale_cogs'] / margin['quantity']), 0)
margin['avg_price'] = round(margin['sale'] / margin['quantity'], 0)
margin['diff_price%'] = ((margin['price_new'] / margin['avg_price'] - 1) * 100)

margin.reset_index()[['name_x', 'price_new', 'avg_price', 'diff_price%']].round(1)

,name_x,price_new,avg_price,diff_price%
0,Кольцо_21111111111,1256.0,1241.0,1.2


<div style="border:solid green 2px; padding: 20px">

Если мы хотим увеличить у позиции Кольцо_21111111111 маржу на 5\%, то цена продажи должна быть 1256р. Это на 1.2\% выше текущей средней цены продажи.

</div>